<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_dl_intro/4%20-%20Usando%20imagens%20do%20mundo%20real/13%20-%20C1_W4_Lab_1_image_generator_no_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow para DeepLearning.AI](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Treinando com ImageDataGenerator

Neste laboratório, você criará e treinará um modelo no conjunto de dados [Horses or Humans](https://www.tensorflow.org/datasets/catalog/horses_or_humans).

Ele contém mais de mil imagens de cavalos e humanos com poses e tamanhos de arquivo variados.

Você usará a classe [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) para preparar esse conjunto de dados para que ele possa ser alimentado em uma rede neural convolucional.

Execute o código abaixo para fazer o download do conjunto de dados compactado `horse-or-human.zip`.

In [ ]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

Em seguida, você pode descompactar o arquivo usando o módulo [zipfile](https://docs.python.org/3/library/zipfile.html).

In [ ]:
import zipfile

# Descompactar o conjunto de dados
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

O conteúdo do .zip é extraído para o diretório base `./horse-or-human`, que, por sua vez, contém subdiretórios `horses` e `humans`.

Resumindo: o conjunto de treinamento são os dados usados para informar ao modelo de rede neural que "é assim que um cavalo se parece" e "é assim que um humano se parece".

Um aspecto a ser observado nesse exemplo: Não rotulamos explicitamente as imagens como cavalos ou humanos. Em vez disso, você usará a API ImageDataGenerator, que é codificada para rotular automaticamente as imagens de acordo com os nomes e a estrutura do diretório. Assim, por exemplo, você terá um diretório de "treinamento" contendo um diretório de "cavalos" e um de "humanos". O `ImageDataGenerator` rotulará as imagens adequadamente para você, reduzindo uma etapa de codificação. 

Agora você pode definir cada um desses diretórios:

In [ ]:
import os

# Diretório com nossas fotos de cavalos de treinamento
train_horse_dir = os.path.join('./horse-or-human/horses')

# Diretório com nossas imagens humanas de treinamento
train_human_dir = os.path.join('./horse-or-human/humans')

Agora veja como são os nomes dos arquivos nos diretórios de treinamento `horses` e `humans`:

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])

train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])

Você também pode descobrir o número total de imagens de cavalos e humanos nos diretórios:

In [ ]:
print('total de imagens de de cavalos para treinamento:', len(os.listdir(train_horse_dir)))
print('total de imagens de humanos para treinamento:', len(os.listdir(train_human_dir)))

Agora, dê uma olhada em algumas imagens para ter uma noção melhor de como elas são. Primeiro, configure os parâmetros do `matplotlib`:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parâmetros para nosso gráfico; produziremos imagens em uma configuração 4x4
nrows = 4
ncols = 4

# Índice para iteração de imagens
pic_index = 0

Agora, exiba um lote de 8 imagens de cavalos e 8 imagens de humanos.

Você pode executar novamente a célula para ver um novo lote a cada vez:

In [ ]:
# Configure o matplotlib fig e dimensione-o para caber em fotos 4x4
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_horse_pix = [os.path.join(train_horse_dir, fname) 
                for fname in train_horse_names[pic_index-8:pic_index]]
next_human_pix = [os.path.join(train_human_dir, fname) 
                for fname in train_human_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_horse_pix+next_human_pix):
  # Configure o subplot; os índices do subplot começam em 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off')  # Não mostre os eixos (ou linhas de grade)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


## Criando um modelo pequeno do zero

Agora você pode definir a arquitetura do modelo que será treinado.

A etapa 1 será a importação do tensorflow.

In [ ]:
import tensorflow as tf

Em seguida, você adiciona camadas convolucionais como no exemplo anterior e achata o resultado final para alimentar as camadas densamente conectadas.

Observe que, como esse é um problema de classificação de duas classes, ou seja, um *problema de classificação binária*, você terminará sua rede com uma ativação [*sigmoidal*](https://wikipedia.org/wiki/Sigmoid_function). Isso faz com que o valor de saída de sua rede seja um único escalar entre 0 e 1, codificando a probabilidade de a imagem atual ser da classe 1 (em oposição à classe 0).

In [ ]:
model = tf.keras.models.Sequential([
    # Observe que a forma de entrada é o tamanho desejado da imagem 300x300 com 3 bytes de cor
    # Essa é a primeira convolução
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # A segunda convolução
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A terceira convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A quarta convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A quinta convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Achatar os resultados para alimentar um DNN
    tf.keras.layers.Flatten(),
    # Camada oculta de 512 neurônios
    tf.keras.layers.Dense(512, activation='relu'),
    # Apenas 1 neurônio de saída. Ele conterá um valor de 0 a 1, sendo 0 para uma classe ("cavalos") e 1 para a outra ("humanos")    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Você pode revisar a arquitetura da rede e as formas de saída com `model.summary()`.

In [ ]:
model.summary()

A coluna "output shape" mostra como o tamanho do mapa de recursos evolui em cada camada sucessiva. Como você viu em uma lição anterior, as camadas de convolução removem os pixels mais externos da imagem, e cada camada de pooling reduz as dimensões pela metade.

Em seguida, você configurará as especificações para o treinamento do modelo. Você treinará o modelo com a perda [`binary_crossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy) porque se trata de um problema de classificação binária e a ativação final é um sigmoide. (Para uma atualização sobre métricas de perda, consulte este [Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/descending-into-ml/video-lecture).) Você usará o otimizador `rmsprop` com uma taxa de aprendizado de `0,001`. Durante o treinamento, você desejará monitorar a acurácia da classificação.

**NOTA**: Nesse caso, o uso do [algoritmo de otimização RMSprop] (https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) é preferível ao [stochastic gradient descent](https://developers.google.com/machine-learning/glossary/#SGD) (SGD), porque o RMSprop automatiza o ajuste da taxa de aprendizado para nós. (Outros otimizadores, como [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) e [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad), também adaptam automaticamente a taxa de aprendizado durante o treinamento e funcionariam igualmente bem aqui).

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

### Pré-processamento de dados

A próxima etapa é configurar os geradores de dados que lerão as imagens nas pastas de origem, convertê-las em tensores `float32` e alimentá-las (com seus rótulos) ao modelo.Você terá um gerador para as imagens de treinamento e outro para as imagens de validação.Esses geradores produzirão lotes de imagens de tamanho 300x300 e seus rótulos (binários).

Como você já deve saber, os dados que entram nas redes neurais geralmente devem ser normalizados de alguma forma para torná-los mais fáceis de serem processados pela rede (ou seja, não é comum alimentar uma ConvNet com pixels brutos). Nesse caso, você pré-processará as imagens normalizando os valores de pixel para que fiquem no intervalo `[0, 1]` (originalmente todos os valores estão no intervalo `[0, 255]`).

No Keras, isso pode ser feito por meio da classe `keras.preprocessing.image.ImageDataGenerator` usando o parâmetro `rescale`.Essa classe `ImageDataGenerator` permite que você instancie geradores de lotes de imagens aumentadas (e seus rótulos) por meio de `.flow(data, labels)` ou `.flow_from_directory(directory)`.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Todas as imagens serão redimensionadas em 1,/255
train_datagen = ImageDataGenerator(rescale=1/255)

# Imagens de treinamento de fluxo em lotes de 128 usando o gerador train_datagen
train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  # Esse é o diretório de origem das imagens de treinamento
        target_size=(300, 300),  # Todas as imagens serão redimensionadas para 300x300
        batch_size=128,
        # Como usamos a perda binary_crossentropy, precisamos de rótulos binários
        class_mode='binary')


### Treinamento

Você pode iniciar o treinamento para 15 épocas - isso pode levar alguns minutos para ser executado.

Observe os valores por época.

A "perda" e a "acurácia" são ótimos indicadores do progresso do treinamento. O `loss` mede a previsão do modelo atual em relação aos rótulos conhecidos, calculando o resultado. A "acurácia", por outro lado, é a porção de suposições corretas. 

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1)

### Previsão do modelo

Agora, dê uma olhada na execução de uma previsão usando o modelo. Esse código permitirá que você escolha um ou mais arquivos do seu sistema de arquivos, carregue-os e execute-os por meio do modelo, fornecendo uma indicação de que o objeto é um cavalo ou um humano.

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
 
   # Previsão de imagens
  path = '/content/' + fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
    
  if classes[0]>0.5:
    print(fn + " é um humano")
  else:
    print(fn + " é um cavalo")
 

### Visualizando representações intermediárias

Para ter uma ideia do tipo de recursos que sua CNN aprendeu, uma coisa divertida a se fazer é visualizar como uma entrada é transformada à medida que passa pelo modelo.

Você pode escolher uma imagem aleatória do conjunto de treinamento e, em seguida, gerar uma figura em que cada linha é a saída de uma camada e cada imagem na linha é um filtro específico nesse mapa de recursos de saída. Execute novamente essa célula para gerar representações intermediárias para uma variedade de imagens de treinamento.

In [ ]:
import numpy as np
import random
from tensorflow.keras.utils import img_to_array, load_img

# Defina um novo modelo que receberá uma imagem como entrada e produzirá
# representações intermediárias para todas as camadas do modelo anterior após
# a primeira.

successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# Prepare uma imagem de entrada aleatória do conjunto de treinamento.
horse_img_files = [os.path.join(train_horse_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)

img = load_img(img_path, target_size=(300, 300))  # esta é uma imagem PIL
x = img_to_array(img) # Matriz Numpy com formato (300, 300, 3)
x = x.reshape((1,) + x.shape)   # Matriz Numpy com formato (1, 300, 300, 3)

# Escalar de 1/255
x /= 255

# Execute a imagem na rede, obtendo assim todas as
# representações intermediárias para essa imagem.
successive_feature_maps = visualization_model.predict(x)

# Esses são os nomes das camadas, para que você possa tê-las como parte do gráfico
layer_names = [layer.name for layer in model.layers[1:]]

# Exibir as representações
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:

    # Faça isso apenas para as camadas conv / maxpool, não para as camadas totalmente conectadas
    n_features = feature_map.shape[-1]  # number of features in feature map

    # Faça isso apenas para as camadas conv / maxpool, não para as camadas totalmente conectadas
    size = feature_map.shape[1]
    
    # Colocar as imagens em mosaico nessa matriz
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
    
      # Colocar cada filtro em uma grade horizontal grande
      display_grid[:, i * size : (i + 1) * size] = x
    
    # Exibir a grade
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

Você pode ver acima como os pixels destacados se transformam em representações cada vez mais abstratas e compactas, especialmente na grade inferior. 

As representações a jusante começam a destacar aquilo a que a rede presta atenção e mostram cada vez menos recursos sendo "ativados"; a maioria é definida como zero.

Isso é chamado de _esparsidade de representação_ e é um recurso fundamental da aprendizagem profunda.

Essas representações carregam cada vez menos informações sobre os pixels originais da imagem, mas informações cada vez mais refinadas sobre a classe da imagem.

Você pode pensar em uma convnet (ou em uma rede profunda em geral) como um pipeline de destilação de informações em que cada camada filtra os recursos mais úteis.

## Limpar

Você continuará com um exercício semelhante no próximo laboratório, mas, antes disso, execute a seguinte célula para encerrar o kernel e liberar recursos de memória:

In [ ]:
from google.colab import runtime
runtime.unassign()